<a href="https://colab.research.google.com/github/Urdzik/Colab/blob/main/MDDL_Urdzik_CS26_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet datasets

In [6]:
import datasets

In [8]:
yelp_polarity  = datasets.load_dataset("yelp_polarity")

Generating test split: 100%|██████████| 38000/38000 [00:00<00:00, 1741687.36 examples/s]


In [9]:
yelp_polarity

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})

In [10]:
print(yelp_polarity.keys())

dict_keys(['train', 'test'])


In [12]:
train_data = yelp_polarity['train']
# test_data = yelp_polarity['test']

In [13]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 560000
})

In [14]:
train_data[100]["text"]

"In general I do like Shake N' Steak, but this location is a hit or miss location!  You never know what kind of quality or service you're going to find here.  A friend and myself went a few weeks back after a movie and it had to be one of the worst trips there EVER!  You can't entirely blame the waitress since she was the only one there for the entire place...poor scheduling on the manager's part. However, while she can't be accountable for the slooooow service, she was accountable for both orders being incorrect.  The burgers were over cooked and the fries were soggie and the milkshake was runny at best...\\n\\nBy far my worst visit to Steak n' Shake!"

In [15]:
!pip install --quiet spacy

In [16]:
import spacy
import tqdm
from typing import List, Tuple

spacy_nlp = spacy.blank("en")

def tokenize_spacy(text: str) -> List[str]:
  """Tokenize string with SpaCy. """

  tokens = spacy_nlp.tokenizer(text)
  return [str(token) for token in tokens]


def tokenize(text: str) -> List[str]:
  return tokenize_spacy(text)


tokenize_spacy("I can't find my $25! oleksiy@google.com http://google.com/ <b>")

['I',
 'ca',
 "n't",
 'find',
 'my',
 '$',
 '25',
 '!',
 'oleksiy@google.com',
 'http://google.com/',
 '<',
 'b',
 '>']

In [17]:
!pip install pystemmer

In [18]:
import Stemmer
from typing import List

def stem(tokens: List[str]) -> List[str]:
  """Lower-case and stem tokens. """

  stemmer = Stemmer.Stemmer("english")
  tokens = [tok.lower() for tok in tokens]
  return stemmer.stemWords(tokens)

example_tokens = tokenize(train_data[2]["text"])
stemmed_tokens = stem(example_tokens)


i do n't know what dr. goldberg was like befor   move to arizona , but let me tell you , stay away from this doctor and this offic . i was go to dr. johnson befor he left and goldberg took over when johnson left . he is not a care doctor . he is onli interest in the co - pay and have you come in for medic refil everi month . he will not give refil and could less about patient 's financi situat . tri to get your 90 day mail away pharmaci prescript through this guy is a joke . and to make matter even wors , his offic staff is incompet . 90 % of the time when you call the offic , they ll put you through to a voic mail , that no one ever answer or return your call . both my adult children and husband have decid to leav this practic after experienc such frustrat . the entir offic has an attitud like they are do you a favor . give me a break ! stay away from this doc and the practic . you deserv better and they will not be there when you realli need them . i have never felt compel to write a

In [19]:
for word in ("sing", "sings", "singing", "singer", "singers"):
  stemmed = stem([word])[0]
  print(f"{word:<20}   =>   {stemmed}")

sing                   =>   sing
sings                  =>   sing
singing                =>   sing
singer                 =>   singer
singers                =>   singer


In [ ]:
from tqdm import tqdm

all_tokens = []
all_tokens_stemmed = []
for doc in tqdm(train_data):
  doc_tokens = tokenize(doc["text"])
  all_tokens += doc_tokens
  all_tokens_stemmed += stem(doc_tokens)

print("Original unique tokens: {:,}".format(len(set(all_tokens))))
print("Stemmed  unique tokens: {:,}".format(len(set(all_tokens_stemmed))))

  8%|▊         | 45980/560000 [00:10<01:44, 4898.09it/s]